## Predicting wine quality given chemical characteristics of the wine
by Yixuan Gao, Bryan Lee, Wangkai Zhu, Timothy Singh

# Summary

# Introduction:


# Methods & Results:


# Discussion:


# References